In [ ]:
# -*- coding: utf-8 -*-
"""
Automated Data Processing Pipeline
Main Script: run_pipeline.py

Flow: Load → Clean → Visualize → Summarize → Export
"""

import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
import logging

# Configure logging to track pipeline execution
logging.basicConfig(filename='data_pipeline.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def load_data(input_path):
    
    try:
        df = pd.read_csv(input_path)
        logging.info(f"Data loaded successfully. Found {len(df)} rows.")
        return df
    except Exception as e:
        logging.error(f"Error loading data: {str(e)}")
        raise

def clean_data(df):
    
    try:
        # Handle missing values by dropping rows with NA
        df_cleaned = df.dropna()
        
        # Clean string columns by stripping whitespace
        for col in df_cleaned.select_dtypes(include=['object']):
            df_cleaned[col] = df_cleaned[col].str.strip()
        
        logging.info(f"Data cleaned. {len(df_cleaned)} clean rows remaining.")
        return df_cleaned
    except Exception as e:
        logging.error(f"Error cleaning data: {str(e)}")
        raise

def visualize_data(df, output_dir):
    
    try:
        os.makedirs(output_dir, exist_ok=True)
        
        # Create histograms for all numeric columns
        numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
        for col in numeric_cols:
            plt.figure()
            df[col].hist(bins=20)
            plt.title(f'Distribution of {col}')
            plt.savefig(os.path.join(output_dir, f'{col}_histogram.png'))
            plt.close()
        
        logging.info(f"Visualizations saved to {output_dir}")
    except Exception as e:
        logging.error(f"Error in visualization: {str(e)}")
        raise

def summarize_data(df, output_dir):
   
    try:
        os.makedirs(output_dir, exist_ok=True)
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        
        # Generate text summary report
        txt_report = f"""Data Summary Report\nDate: {current_time}\n\n"""
        txt_report += df.describe().to_string()
        
        with open(os.path.join(output_dir, f'summary_{current_time}.txt'), 'w') as f:
            f.write(txt_report)
        
        # Generate CSV summary report
        summary_df = df.describe().reset_index()
        summary_df.to_csv(os.path.join(output_dir, f'summary_{current_time}.csv'), index=False)
        
        logging.info(f"Summary reports saved to {output_dir}")
    except Exception as e:
        logging.error(f"Error generating summaries: {str(e)}")
        raise

def export_results(df, output_path):
    
    try:
        df.to_csv(output_path, index=False)
        logging.info(f"Cleaned data exported to {output_path}")
    except Exception as e:
        logging.error(f"Error exporting data: {str(e)}")
        raise

def run_pipeline(input_file, output_base_dir):

    try:
        
        current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        output_dir = os.path.join(output_base_dir, f"run_{current_time}")
        viz_dir = os.path.join(output_dir, "visualizations")
        summary_dir = os.path.join(output_dir, "summaries")
        
        logging.info("Starting pipeline execution...")
        
        
        raw_data = load_data(input_file)
        
       
        cleaned_data = clean_data(raw_data)
        
        visualize_data(cleaned_data, viz_dir)
        
    
        summarize_data(cleaned_data, summary_dir)
        
        export_results(cleaned_data, os.path.join(output_dir, "cleaned_data.csv"))
        
        logging.info("Pipeline completed successfully!")
        print(f"Processing complete! Results saved to {output_dir}")
    except Exception as e:
        logging.error(f"Pipeline failed: {str(e)}")
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    INPUT_DATA = "Sample DataFrame with missing values.csv"  
    OUTPUT_BASE_DIR = "output"   
    
    run_pipeline(INPUT_DATA, OUTPUT_BASE_DIR)